# advent of code 2024 - [day 8](https://adventofcode.com/2024/day/8)

### Parsing

In [ ]:
def gen_lists(file='input.txt'):
    """Generates tuples of integers"""
    file = open(file, 'r')
    for ix, line in enumerate(file):
        for jx, c in enumerate(line.strip()):
            yield ix, jx, c

In [ ]:
filename = "input.txt"
#filename = "test.txt"

In [ ]:
from functools import reduce
import doctest

## Graph Solution

In [ ]:
import os

NEO4J_URI = os.environ['NEO4J_URI']
NEO4J_USERNAME = os.environ['NEO4J_USERNAME']
NEO4J_PASSWORD = os.environ['NEO4J_PASSWORD']

from graphdatascience import GraphDataScience
gds = GraphDataScience(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

# Check the installed GDS version on the server
print(gds.version())
assert gds.version()

from neo4j import GraphDatabase
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

import pandas as pd

def infer (rules, params={}):
    """
    This is a function you can use if you want to run a set of inference rules
    until a convergence is reached. why not use it in a RDF-like reasoning context?
    """
    counter = 0
    while True:
        counter += 1
        any_update = False
        for rule in rules:
            with driver.session(database="neo4j") as session:
                result = session.run(rule, params)
            any_update = any_update or result.consume().counters._contains_updates
        if not any_update:
            break

In [ ]:
# cleaning
def clean():
    queries = [
    "CALL apoc.schema.assert({},{});",
    """MATCH (n)
    CALL {WITH n DETACH DELETE n}
    IN TRANSACTIONS OF 1000 ROWS;""",
    """CALL gds.graph.list()
    YIELD graphName
    WITH graphName AS g
    CALL (g) {CALL gds.graph.drop(g) YIELD graphName RETURN graphName}
    WITH graphName RETURN graphName;"""]

    for q in queries:
        gds.run_cypher(q, {})

### Ingestion


In [ ]:
def ingest(filename):

    tiles = [{'row':ix, 'col':jx, 'val':c} for ix, jx, c in list(gen_lists(filename))]
    
    clean()

    gds.run_cypher('CREATE INDEX tile_col_row IF NOT EXISTS FOR (t:Tile) ON (t.col, t.row)')
    gds.run_cypher('CREATE INDEX tile_val IF NOT EXISTS FOR (t:Tile) ON (t.val)')
    
    query_ingest = """
    UNWIND $tiles AS tile
    CREATE (:Tile {row:tile.row, col:tile.col, val:tile.val} )
    """

    gds.run_cypher(query_ingest, {"tiles":tiles})

## part 1

In [ ]:
part1_build_queries = ["""MATCH (t:Tile)
SET t:InMap""",
"""MATCH (t:Tile)
WHERE t.val <> '.'
SET t:Antenna""",
"""MATCH (a1:Tile:Antenna), (a2:Tile:Antenna)
WHERE a1.val = a2.val
AND a1 < a2
MERGE (a1)-[:SAME_FREQUENCY]->(a2)""",
"""MATCH (a1:Antenna)-[:SAME_FREQUENCY]->(a2:Antenna)
WITH a1, a2, {row: a2.row - a1.row, col: a2.col - a1.col} AS v
MERGE (c1:Tile {row : a2.row + v.row, col : a2.col + v.col})
  SET c1:Covered
MERGE (c2:Tile {row : a1.row - v.row, col : a1.col - v.col})
  SET c2:Covered"""]

In [ ]:
ingest(filename)
for q in part1_build_queries:
    gds.run_cypher(q)
gds.run_cypher('MATCH (t:InMap&Covered) RETURN count(t) AS part1')

### Part 2

In [ ]:
part2_build_queries = [
"""MATCH (t:Tile)
WHERE t.val <> '.'
SET t:Antenna""",
"""MATCH (a1:Tile:Antenna), (a2:Tile:Antenna)
WHERE a1.val = a2.val
AND a1 < a2
SET a1:Antinode, a2:Antinode
MERGE (a1)-[:SAME_FREQUENCY]->(a2)""",
"""MATCH (a1:Antenna)-[:SAME_FREQUENCY]->(a2:Antenna)
WITH a1, a2, {row: a2.row - a1.row, col: a2.col - a1.col} AS v
MATCH (c:Tile)
WHERE (
  a1.col <> a2.col
  AND c.col <> a1.col
  AND toFloat(c.row - a1.row)/(c.col - a1.col) = toFloat(a2.row - a1.row)/(a2.col - a1.col)
)
OR
c.col = a1.col = a2.col
SET c:Antinode""",
"""//for viz
MATCH (t:Tile) SET t.X = t.col, t.Y = -t.row
"""]

In [ ]:
ingest(filename)
for q in part2_build_queries:
    gds.run_cypher(q)
gds.run_cypher('MATCH (t:Antinode) RETURN count(t) AS part2')